# 📘 출력 파서(Output Parser)

---

## 01. 출력파서(Output Parser)

### 📌 개념
- LLM 출력은 보통 자유 텍스트 → 그대로 쓰기 어려움
- **OutputParser** : LLM 출력 → 구조화된 데이터(문자열, JSON, 리스트 등) 변환
- 장점:
  - 다운스트림 로직과 쉽게 연결
  - 안정적인 파싱 (에러 줄임)

In [2]:
### 💻 예시 코드
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:10]) 

sk-proj-73


In [3]:
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

parser = StrOutputParser()

chain = llm | parser
print(chain.invoke("Answer in one sentence: What is LangChain?"))

LangChain is a decentralized language learning platform that uses blockchain technology to connect language learners with native speakers for personalized lessons.


## 02. PydanticOutputParser
### 📌 개념

- Pydantic 모델을 기반으로 LLM 출력 파싱

- 데이터 유효성 검사 자동 수행

In [4]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate

# 데이터 스키마 정의
class Person(BaseModel):
    name: str = Field(description="사람 이름")
    age: int = Field(description="나이")

parser = PydanticOutputParser(pydantic_object=Person)

prompt = PromptTemplate(
    template="다음 정보를 JSON으로 작성해줘:\n이름: {name}\n나이: {age}\n{format_instructions}",
    input_variables=["name", "age"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser
print(chain.invoke({"name": "철수", "age": "25"}))

name='철수' age=25


## 03. with_structured_output() 바인딩
### 📌 개념

- LLM을 직접 Pydantic 모델에 바인딩

- 별도의 파서 연결 필요 없음

In [5]:
llm_structured = llm.with_structured_output(Person)
print(llm_structured.invoke("이름은 영희이고 나이는 30살인 사람의 JSON을 만들어줘"))

C:\Users\james\anaconda3\envs\my_new_env\lib\site-packages\langchain_openai\chat_models\base.py:1927: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


name='영희' age=30


## 04. LangSmith 에서 OutputParser 흐름 확인
### 📌 개념

- LangSmith 대시보드에서 체인 실행 과정 추적

- LLM → OutputParser로 변환되는 과정 시각화

## 05. 콤마로 구분된 리스트 출력파서 (CommaSeparatedListOutputParser)
### 📌 개념

- 문자열을 리스트로 변환

- 예: "사과, 배, 포도" → ["사과", "배", "포도"]

In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate.from_template("과일을 3개 나열해줘. {format_instructions}")
chain = prompt.partial(format_instructions=parser.get_format_instructions()) | llm | parser

print(chain.invoke({}))

['사과', '바나나', '오렌지']


## 06. 구조화된 출력파서(StructuredOutputParser)
### 📌 개념

- JSON Schema 기반 구조화된 출력 생성

- 키:값 형태 데이터 추출에 적합

In [7]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schemas = [
    ResponseSchema(name="name", description="이름"),
    ResponseSchema(name="hobby", description="취미")
]

parser = StructuredOutputParser.from_response_schemas(schemas)

prompt = PromptTemplate(
    template="사람의 이름과 취미를 알려줘.\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | llm | parser
print(chain.invoke({}))

{'name': '영희', 'hobby': '독서'}


## 07. JSON 형식 출력파서 (JsonOutputParser)
### 📌 개념

- LLM 출력 → JSON으로 파싱

In [11]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

parser = JsonOutputParser()

prompt = PromptTemplate.from_template(
    "책 1권의 제목과 저자를 JSON으로 제공해줘.\n{format_instructions}"
)

chain = prompt.partial(format_instructions=parser.get_format_instructions()) | llm | parser
print(chain.invoke({}))

{'title': 'The Great Gatsby', 'author': 'F. Scott Fitzgerald'}


## 08. Pandas DataFrame 출력파서 (PandasDataFrameOutputParser)
### 📌 개념

- LLM 출력 → Pandas DataFrame으로 변환

- 데이터 분석에 활용

In [15]:
# from langchain_experimental.output_parsers import PandasDataFrameOutputParser

# parser = PandasDataFrameOutputParser()

# prompt = PromptTemplate.from_template(
#     "3명의 이름과 나이를 표로 작성해줘.\n{format_instructions}"
# )

# chain = prompt.partial(format_instructions=parser.get_format_instructions()) | llm | parser
# df = chain.invoke({})
# print(df)
# print(type(df))

## 09. 날짜형식 출력파서 (DatetimeOutputParser)
### 📌 개념

- 문자열을 datetime 객체로 파싱

In [10]:
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser()

prompt = PromptTemplate.from_template("오늘 날짜를 {format_instructions}")
chain = prompt.partial(format_instructions=parser.get_format_instructions()) | llm | parser

print(chain.invoke({}))

2022-11-30 15:45:23.123456


## 10. 열거형 출력파서 (EnumOutputParser)
### 📌 개념

- LLM 출력 → Enum 타입으로 제한

- 특정 선택지 중 하나만 받도록 강제

In [16]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

class Sentiment(Enum):
    positive = "positive"
    negative = "negative"
    neutral = "neutral"

parser = EnumOutputParser(enum=Sentiment)

prompt = PromptTemplate.from_template(
    "다음 문장의 감정을 분류해줘: {sentence}\n{format_instructions}"
)

chain = prompt.partial(format_instructions=parser.get_format_instructions()) | llm | parser
print(chain.invoke({"sentence": "오늘은 기분이 좋다!"}))

Sentiment.positive


## ✅ 정리

- OutputParser = 자유 텍스트를 구조화 데이터로 변환

### 주요 파서:

- StrOutputParser → 단순 문자열

- PydanticOutputParser → Pydantic 모델 기반

- CommaSeparatedListOutputParser → 리스트

- StructuredOutputParser / JsonOutputParser → JSON

- PandasDataFrameOutputParser → DataFrame

- DatetimeOutputParser → 날짜

- EnumOutputParser → Enum 값

- LangSmith에서 파싱 흐름 추적 가능